# Рекомендация тарифов
___
# Tariff recommendation

В доступе имеются данные о поведении клиентов, которые перешли на определенные тарифы. Необходимо создать модель для задач классификации, с помощью которой впоследствии можно производить выбор наиболее подходящего тарифа.

Нужно построить модель с наибольшим значением **accuracy**. Минимальное целевое значение параметра - 0,75.
___
Access has data on the behavior of customers who have switched to certain tariffs. It is necessary to create a model for classification tasks, with the help of which you can subsequently select the most appropriate tariff.

You need to build the model with the highest **accuracy** value. The minimum target value of the parameter is 0.75.

## Исследование данных
___
## Data exploration

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.dummy import DummyClassifier

In [2]:
df = pd.read_csv('users_behavior.csv')

In [3]:
df.head(5)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
df.duplicated().mean()

0.0

In [6]:
df.isna().mean()*100

calls       0.0
minutes     0.0
messages    0.0
mb_used     0.0
is_ultra    0.0
dtype: float64

Данные без дубликатов и пропусков, типы данных корректны.
___
Data without duplicates and gaps, data types are correct.

## Предобработка данных
___
## Data preprocessing

In [7]:
df_test, df_t_v = train_test_split(df, test_size=0.4, random_state=12345)

In [8]:
df_train, df_valid = train_test_split(df_t_v, test_size=0.5, random_state=12345) 

Получилось 3 выборки, **df_test** (50% от изначальной df), **df_train** и **df_valid** (внутреннее соотношение 75% и 25%).
___
This resulted in 3 samples, **df_test** (50% of the original df), **df_train** and **df_valid** (internal ratio of 75% and 25%).

## Исследование моделей
___
## Exploring Models

In [9]:
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']

In [10]:
print(features_train.shape)
print(target_train.shape)
print(features_valid.shape)
print(target_valid.shape)

(643, 4)
(643,)
(643, 4)
(643,)


Данные структурированы по признаку **is_ultra**.
___
The data is structured according to the attribute **is_ultra**.

Пробуем решающее дерево.
___
Let's try a decision tree.

In [11]:
best_model_dt = None
best_result_dt = 0
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
    model.fit(features_train, target_train) # обучите модель на тренировочной выборке
    result = model.score(features_valid, target_valid) # посчитайте качество модели на валидационной выборке
    if result > best_result_dt:
        best_model_dt = model # сохраните наилучшую модель
        best_result_dt = result #  сохраните наилучшее значение метрики accuracy на валидационных данных
        
print("Accuracy лучшей модели:", best_model_dt)
print('Лучшая модель -', best_result_dt)

Accuracy лучшей модели: DecisionTreeClassifier(max_depth=4, random_state=12345)
Лучшая модель - 0.7853810264385692


Доля правильных ответов выше обозначенного значения, значит данная модель нам подходит.
___
The proportion of correct answers is higher than the indicated value, which means that this model suits us.

Пробуем случайный лес.
___
Let's try a random forest.

In [12]:
best_model_rf = None
best_result_rf = 0
for est in range(1, 16):
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result_rf:
        best_model_rf = model 
        best_result_rf = result 
        
print("Accuracy лучшей модели:", best_model_rf)
print('Лучшая модель -', best_result_rf)

Accuracy лучшей модели: RandomForestClassifier(n_estimators=14, random_state=12345)
Лучшая модель - 0.7978227060653188


Доля правильных ответов выше обозначенного значения, больше данного значения в случае использования **решающего дерева**.
___
The proportion of correct answers is higher than the indicated value, more than this value in the case of using **decision tree**.

Пробуем логистическую регрессию.
___
Let's try logistic regression.

In [13]:
best_model_lr = None
best_result_lr = 0
for iter in range(60, 121):
    model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=iter)
    model.fit(features_train, target_train)
    result = model.score(features_valid, target_valid)
    if result > best_result_lr:
        best_model_lr = model 
        best_result_lr = result 
        
print("Accuracy лучшей модели:", best_result_lr)
print('Лучшая модель -', best_model_lr)

Accuracy лучшей модели: 0.6889580093312597
Лучшая модель - LogisticRegression(max_iter=60, random_state=12345)


Доля правильных ответов менее 0.75, данный вариант нам не подходит.
___
The proportion of correct answers is less than 0.75, this option does not suit us.

## Проверка модели на тестовой выборке
___
## Validate the model on the test set

In [14]:
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']

In [15]:
model = RandomForestClassifier(n_estimators=15, random_state=12345)
model.fit(features_test, target_test)
result = model.score(features_test, target_test)
print(result)

0.9896265560165975


Модель хорошо показала себя на тестовой выборке. Доля правильных ответов более 0.75, равна 0.989626, что является практически идеальным результатом. Соответственно метод "случайный лес" в данном случае является оптимальным, особенно с указанными выше гиперпараметрами (RandomForestClassifier(n_estimators=15, random_state=12345)).
___
The model performed well on the test sample. The proportion of correct answers over 0.75 is 0.989626, which is an almost perfect result. Accordingly, the "random forest" method in this case is optimal, especially with the above hyperparameters (RandomForestClassifier(n_estimators=15, random_state=12345)).